In [ ]:
# Library loads
import numpy as np
import sklearn
import pandas as pd
import os

from numpy import random
from sklearn import preprocessing
from sklearn import tree
from sklearn import ensemble
from sklearn import svm

from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
import xgboost as xgb
from sklearn.neural_network import MLPClassifier

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.preprocessing import Normalizer
#from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix
from sklearn.metrics import (precision_score, recall_score, f1_score, accuracy_score, mean_squared_error,mean_absolute_error, roc_curve, classification_report, auc)
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# For creating plots using data 
'''
import seaborn as sns
import plotly
from plotly import __version__
print(plotly.__version__)
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)
cf.go_offline()
import plotly.io as pio
pio.renderers.default = 'colab'
'''
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import RandomOverSampler

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
from numpy.random import seed
seed(1)

seed=1

data_2014 = pd.read_csv('/content/drive/My Drive/Data/2014_Financial_Data.csv', header=0)
data_2015 = pd.read_csv('/content/drive/My Drive/Data/2015_Financial_Data.csv', header=0)
data_2016 = pd.read_csv('/content/drive/My Drive/Data/2016_Financial_Data.csv', header=0)
data_2017 = pd.read_csv('/content/drive/My Drive/Data/2017_Financial_Data.csv', header=0)
data_2018 = pd.read_csv('/content/drive/My Drive/Data/2018_Financial_Data.csv', header=0)
#print(data_2014.shape, data_2015.shape, data_2016.shape, data_2017.shape)
#filename='/content/drive/My Drive/Data/2018_Financial_Data.csv'
#print(filename)

#data=pd.read_csv(filename, header=0)
data_all=pd.concat([data_2014,data_2015,data_2016,data_2017])

data = pd.DataFrame(data_2014)
data_2015 =pd.DataFrame(data_2015)
data.shape, data_2015.shape


((3808, 225), (4120, 225))

**Creating function for Evaluation using Accuracy, Precision**

In [ ]:
print(data.iloc[:, -1].unique().size)

if data.iloc[:, -1].unique().size==2:
    def evaluate(y_test, y_pred, y_scores):
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='binary', pos_label=1) #labels=np.unique(y_pred))
        recall = recall_score(y_test, y_pred, average='binary', pos_label=1) #labels=np.unique(y_pred))
        f1 = f1_score(y_test, y_pred, average='binary', pos_label=1) #labels=np.unique(y_pred))
        auc = roc_auc_score(y_test, y_scores)
        return [accuracy, precision, recall, f1, auc]
else:

    def evaluate(y_test, y_pred, y_scores):
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted', pos_label=1) #labels=np.unique(y_pred))
        recall = recall_score(y_test, y_pred, average='weighted', pos_label=1) #labels=np.unique(y_pred))
        f1 = f1_score(y_test, y_pred, average='weighted', pos_label=1) #labels=np.unique(y_pred))
				#metrics.f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))		 

        auc = roc_auc_score(y_test, y_scores)
        return [accuracy, precision, recall, f1, auc]

2


In [ ]:
data.fillna(0, inplace=True)
data_2015.fillna(0, inplace=True)

**Replace all NaN with 0 and blanks with 0 Enable coding to convert Sector**

In [ ]:
#data.fillna(0, inplace=True)
#data_2016.fillna(0, inplace=True)

# Categorical boolean mask
categorical_feature_mask = data.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = data.columns[categorical_feature_mask].tolist()
#categorical_cols1 = data_2018.columns[categorical_feature_mask].tolist()

print(categorical_cols)
#Use LabelEncoder() to transfer categorical data to numurical data
le=LabelEncoder()
print(len(categorical_cols))
if len(categorical_cols)!=0:
    data[categorical_cols] = data[categorical_cols].apply(lambda col: le.fit_transform(col))
    #data_2018[categorical_cols] = data_2018[categorical_cols].apply(lambda col: le.fit_transform(col))
#data[categorical_cols].head(5)
#data_2018[categorical_cols1].head(5)

data=np.array(data)
data_2015=np.array(data_2015)
#If first column is ID, then manuly removed from the dataset





['Unnamed: 0', 'Sector']
2


**Define Train and Test data and Label data**

In [ ]:
#X,y=data[:,1:-4], data[:, -1]
#X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.20, random_state=seed)
#y_train=y_train.astype(int)

X_train,y_train=data[:,1:-4], data[:, -1]
X_test,y_test=data_2015[:,1:-4], data_2015[:, -1]
#ros = RandomOverSampler(random_state=0)

#X_train, y_train = ros.fit_resample(X_train, y_train)


scaler = preprocessing.StandardScaler().fit(X_train)
X_train2 = scaler.transform(X_train)
X_test2 = scaler.transform(X_test)



traindata = np.array(X_train2)
trainlabel = np.array(y_train)

testdata = np.array(X_test2)
testlabel = np.array(y_test)

traindata=traindata.astype(float)
trainlabel=trainlabel.astype(int)
testdata=testdata.astype(float)
testlabel = testlabel.astype(int)


**MLP**

In [ ]:
def MLP(X_train,y_train,X_test,y_test):
    clf = MLPClassifier(solver='lbfgs', activation= 'relu', alpha=1e-5,hidden_layer_sizes=(10, 5), random_state=seed)
    
    l=X_train[0].size
    lr = np.logspace(-5,-1, 5)
    #print(lr)
    #print(X_train[0].size)
    hz=[(10, 5),(10,5,3)]
    param_grid = {'alpha': lr,'hidden_layer_sizes':hz}

    gridcv = sklearn.model_selection.GridSearchCV(clf, param_grid, verbose=1, cv=3)
    gridcv.fit(X_train, y_train)
					 

    gridcv = gridcv.fit(X_train,y_train)
    y_pred = gridcv.best_estimator_.predict(X_test)
    y_scores = gridcv.best_estimator_.predict_proba(X_test)[:,1]

    print("best parameters:", gridcv.best_params_)
    np.savetxt('/content/drive/My Drive/Data/Result/predictedMLP.txt', y_pred, fmt='%01d')
    return evaluate(y_test,y_pred,y_scores)

# Logistic regression

In [ ]:
def LogisticRegression(X_train,y_train,X_test,y_test):
    clf = sklearn.linear_model.LogisticRegression(random_state=seed, solver='lbfgs')
    clf = clf.fit(X_train,y_train)
								   
					 
							 
    y_pred = clf.predict(X_test)
    y_scores = clf.predict_proba(X_test)[:,1]
    np.savetxt('/content/drive/My Drive/Data/Result/predictedLR.txt', y_pred, fmt='%01d')
    return evaluate(y_test,y_pred,y_scores)

In [ ]:
def CART(X_train,y_train,X_test,y_test):
   # clf = tree.DecisionTreeClassifier(max_depth=5)
   # clf = tree.DecisionTreeClassifier(criterion='entropy',max_depth=5,min_samples_leaf=5)

    clf = tree.DecisionTreeClassifier( random_state=seed)

    md=np.arange(1,50,10)
    param_grid = {'max_depth': md}
					 
							 

    gridcv = sklearn.model_selection.GridSearchCV(clf, param_grid, verbose=1, cv=3)
    gridcv.fit(X_train, y_train)

    #clf = clf.fit(X_train,y_train)
    y_pred = gridcv.best_estimator_.predict(X_test)
    y_scores = gridcv.best_estimator_.predict_proba(X_test)[:,1]
    np.savetxt('/content/drive/My Drive/Data/Result/predictedDTbase.txt', y_pred, fmt='%01d')
    print("best parameters:", gridcv.best_params_)
    #tree.plot_tree(clf.fit(X_train,y_train))
    #tree.export_graphviz()
    #dot_data = tree.export_graphviz(clf, out_file=None)
    #graph = graphviz.Source(dot_data)
    #graph.render("tree")
    return evaluate(y_test,y_pred,y_scores)

In [ ]:
#randomforest

def RandomForest(X_train,y_train,X_test,y_test):
    clf = ensemble.RandomForestClassifier(max_features=None,random_state=seed)

    md = np.arange(1, 50, 10)
    ne = np.arange(1, 50, 10)
    param_grid = {'max_depth': md,'n_estimators':ne}
					 
							 

    gridcv = sklearn.model_selection.GridSearchCV(clf, param_grid, verbose=1, cv=3)
    gridcv.fit(X_train, y_train)

    gridcv = gridcv.fit(X_train,y_train)
    y_pred = gridcv.best_estimator_.predict(X_test)
    y_scores = gridcv.best_estimator_.predict_proba(X_test)[:,1]
    print("best parameters:", gridcv.best_params_)
    np.savetxt('/content/drive/My Drive/Data/Result/predictedRFbase.txt', y_pred, fmt='%01d')
    return evaluate(y_test,y_pred,y_scores)

In [ ]:
def svm_one_class(X_train,y_train,X_test,y_test):
    _train = preprocessing.normalize(X_train, norm='l2')
    _test = preprocessing.normalize(X_test, norm='l2')
    lin_clf = svm.LinearSVC()
    lin_clf.fit(X_train, y_train)
    LinearSVC(C=1000, class_weight=None, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=seed, tol=0.0001,
     verbose=0)
    y_pred = lin_clf.predict(X_test)
    y_scores = lin_clf.decision_function(X_test)
    np.savetxt('/content/drive/My Drive/Data/Result/predictedSVMbase.txt', y_pred, fmt='%01d')
    return evaluate(y_test,y_pred,y_scores)

In [ ]:
def SVM(X_train,y_train,X_test,y_test):
	
    svm = sklearn.svm.SVC(kernel='rbf')
    C_grid = np.logspace(0, 3, 4)
    gamma_grid = np.logspace(-2, 1, 4)
    param_grid = {'C': C_grid, 'gamma': gamma_grid}
    gridcv = sklearn.model_selection.GridSearchCV(svm, param_grid, verbose=1, cv=3)
    gridcv.fit(X_train, y_train)

    y_pred = gridcv.best_estimator_.predict(X_test)
    y_scores = gridcv.best_estimator_.decision_function(X_test)
    print("best parameters:", gridcv.best_params_)
    np.savetxt('/content/drive/My Drive/Data/Result/predictedSVMNormbase.txt', y_pred, fmt='%01d')
    return evaluate(y_test,y_pred,y_scores)

In [ ]:
def Gaussian_NB(X_train,y_train,X_test,y_test):
    gnb = GaussianNB()
		   
    y_pred = gnb.fit(X_train, y_train).predict(X_test)
							 

								
    y_scores  = gnb.fit(X_train, y_train).predict_proba(X_test)[:,1]

    np.savetxt('/content/drive/My Drive/Data/Result/predictedNBbase.txt', y_pred, fmt='%01d')
    return evaluate(y_test,y_pred,y_scores)

 

In [ ]:
def kNN(X_train,y_train,X_test,y_test):
    clf = KNeighborsClassifier()
    #n_neighbors=5
    nn=np.arange(1,100,10)
    param_grid = {'n_neighbors': nn}

    gridcv = sklearn.model_selection.GridSearchCV(clf, param_grid, verbose=1, cv=3)
    gridcv.fit(X_train, y_train)
    y_pred = gridcv.best_estimator_.predict(X_test)
    y_scores = gridcv.best_estimator_.predict_proba(X_test)[:,1]
    print("best parameters:", gridcv.best_params_)
    np.savetxt('/content/drive/My Drive/Data/Result/predictedKNNbase.txt', y_pred, fmt='%01d')
    return evaluate(y_test,y_pred,y_scores)

In [ ]:
def AdaBoost(X_train,y_train,X_test,y_test):
   # dt_stump = DecisionTreeClassifier(max_depth=5, min_samples_leaf=1)
    dt_stump = DecisionTreeClassifier()
    dt_stump.fit(X_train, y_train)
    dt_stump_err = 1.0 - dt_stump.score(X_test, y_test)

    clf = ensemble.AdaBoostClassifier(base_estimator=dt_stump,learning_rate=0.1, algorithm='SAMME')
    clf= clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    #y_pred = clf.predict(X_test)
    y_scores = clf.predict_proba(X_test)[:,1]
    print("best parameters:", gridcv.best_params_)
    np.savetxt('/content/drive/My Drive/Data/Result/predictedABbase.txt', y_pred, fmt='%01d')

    return evaluate(y_test,y_pred,y_scores)

In [ ]:
def GBDT(X_train,y_train,X_test,y_test):
    n_estimators = [10,50,100]
    learning_rate = [0.01,0.1,1]
    max_depth=[1,5,10]
    param_grid = {'n_estimators': n_estimators, 'learning_rate': learning_rate,'max_depth':max_depth}

    clf = ensemble.GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=seed)
    #clf=clf.fit(X_train, y_train)

    gridcv = sklearn.model_selection.GridSearchCV(clf, param_grid, verbose=1, cv=3)
    gridcv.fit(X_train, y_train)
    y_pred = gridcv.best_estimator_.predict(X_test)
    y_scores = gridcv.best_estimator_.predict_proba(X_test)[:,1]
    print("best parameters:", gridcv.best_params_)
    np.savetxt('/content/drive/My Drive/Data/Result/predictedBGDTbase.txt', y_pred, fmt='%01d')
    return evaluate(y_test,y_pred,y_scores)

In [ ]:
print("NB training")
a1= Gaussian_NB(traindata,trainlabel,testdata,testlabel)
print(a1)
#print("XG Boost training")
#a2=xgboost(traindata,trainlabel,testdata,testlabel)
#print(a2)




NB training
[0.6851941747572815, 0.7082027168234065, 0.9377378069872017, 0.8069653222205686, 0.5162451073573919]


In [ ]:
print("LR training")
a2=LogisticRegression(traindata,trainlabel,testdata,testlabel)
print(a2)

LR training


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.4475728155339806, 0.8078078078078078, 0.27914216534071257, 0.4149100257069409, 0.5924298888922975]


In [ ]:
print("MLP training")
a3=MLP(traindata,trainlabel,testdata,testlabel)
print(a3)



MLP training
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: Convergenc

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REAC

best parameters: {'alpha': 0.0001, 'hidden_layer_sizes': (10, 5, 3)}
[0.4524271844660194, 0.7883742052679382, 0.30024213075060535, 0.4348697394789579, 0.557925905119533]


In [ ]:
print("RF training")
a5=RandomForest(traindata,trainlabel,testdata,testlabel)
print(a5)

RF training
Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  6.7min finished


Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  6.7min finished


best parameters: {'max_depth': 1, 'n_estimators': 21}
[0.38859223300970874, 0.9366197183098591, 0.13801452784503632, 0.24057883629785953, 0.6300803340464318]


In [ ]:
print("DT training")
a6=CART(traindata,trainlabel,testdata,testlabel)
print(a6)


DT training
Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    6.4s finished


best parameters: {'max_depth': 1}
[0.38859223300970874, 0.9366197183098591, 0.13801452784503632, 0.24057883629785953, 0.5580227236458705]


In [ ]:
print("SVM training")
a4= SVM(traindata,trainlabel,testdata,testlabel)
print(a4)



SVM training
Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:  2.9min finished


best parameters: {'C': 1.0, 'gamma': 0.1}
[0.4186893203883495, 0.824607329842932, 0.2179176755447942, 0.3447332421340629, 0.5737871157620279]


In [ ]:

Result = ['a1','a2', 'a3','a4','a5','a6']

In [ ]:
print("Naive_bayes")
print("[accuracy,precision,recall,f1,auc]")
print('a1')

print("NB training")
a2=Gaussian_NB(traindata,trainlabel,testdata,testlabel)
print(a2)

print("LogisticRegression")
print("[accuracy,precision,recall,f1,auc]")
print(a3)

print("SVM")
print("[accuracy,precision,recall,f1,auc]")
print(a4)


print("MLP")
print("[accuracy,precision,recall,f1,auc]")
print(a5)

print("RandomForest")
print("[accuracy,precision,recall,f1,auc]")
print(a6)

print("CART")
print("[accuracy,precision,recall,f1,auc]")
print(a7)


Naive_bayes
[accuracy,precision,recall,f1,auc]
a1
NB training
[0.6851941747572815, 0.7082027168234065, 0.9377378069872017, 0.8069653222205686, 0.5162451073573919]
LogisticRegression
[accuracy,precision,recall,f1,auc]
[0.4524271844660194, 0.7883742052679382, 0.30024213075060535, 0.4348697394789579, 0.557925905119533]
SVM
[accuracy,precision,recall,f1,auc]
[0.4186893203883495, 0.824607329842932, 0.2179176755447942, 0.3447332421340629, 0.5737871157620279]
MLP
[accuracy,precision,recall,f1,auc]
[0.38859223300970874, 0.9366197183098591, 0.13801452784503632, 0.24057883629785953, 0.6300803340464318]
RandomForest
[accuracy,precision,recall,f1,auc]
[0.38859223300970874, 0.9366197183098591, 0.13801452784503632, 0.24057883629785953, 0.5580227236458705]
CART
[accuracy,precision,recall,f1,auc]


NameError: ignored

In [ ]:
def tsneShow(X,y,Xt,yt):
    yys = np.array(y)
    yyt = np.array(yt)
    for i in range(0, yyt.size):
        if yyt[i] == 0:
            yyt[i] = 2
        elif yyt[i] == 1:
            yyt[i] = 3

    H = np.vstack((X, Xt))
    Y = np.concatenate((yys, yyt), axis=0)

    pca = PCA().fit_transform(H)
    tsne = TSNE(n_components=2, learning_rate=100).fit_transform(H)

    markers = ('.')
    plt.figure(figsize=(12, 6))
    # plt.subplot(121)
    plt.scatter(tsne[:, 0], tsne[:, 1], c=Y, marker=markers[0], cmap=plt.cm.gist_rainbow)

    plt.colorbar()
    plt.show()